<a href="https://colab.research.google.com/github/astrfo/RS_init/blob/main/RS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import sys
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
class Environment(object):

    def __init__(self, K):
        self._K = K
        self._prob = [np.random.rand() for _ in range(K)]   #一様分布からサンプリング

    def play(self, arm):
        if self._prob[arm] > random.random():
            return 1
        else:
            return 0

In [ ]:
class Agent(object):

    def __init__(self, k):
        self._k = k
        self._V = None
        self._n = None
        self._eps = sys.float_info.min  #RS式でゼロ除算を防ぐため

    def initialize(self):
        self._V = np.array([0.5] * self._k)
        self._n = np.array([self._eps] * self._k)

In [2]:
K = 20
arm_prob = np.array([np.random.rand() for _ in range(K)])
arm_prob

array([0.01552502, 0.83617327, 0.38771308, 0.52052895, 0.96019493,
       0.47733024, 0.33209924, 0.38663927, 0.70810371, 0.29002406,
       0.82375415, 0.13382304, 0.68687707, 0.42595111, 0.02935369,
       0.38028751, 0.31425619, 0.06047769, 0.28614631, 0.43704415])

In [4]:
sys.float_info.min

2.2250738585072014e-308